In [ ]:
#Esta é a 1° questão da parte de dados
#Uma observação válida é que essa é a minha primeira vez usando o Júpter / Colab. Achei que seria melhor de enviar em vez de fazer no Vscode até por quê teoricamente neste ambiente o poder de processmento é maior.

import pandas as pd

caminho_do_arquivo = "/content/drive/MyDrive/archive/olist_products_dataset.csv"  #Você que vai inspecionar meu código eu recomendo colocar o caminho de onde esta alocado os arquivos de set!

# Lê o arquivo CSV
set = pd.read_csv(caminho_do_arquivo, usecols=[0,1], encoding="utf-8")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

display(set)

#Neste arquivo eu considerei importante apenas a exibição das colunas de product_id e product_category_name pois é onde a maioria das pessoas iriam procurar.

In [ ]:
#Este codigo aqui para ser sincero era para ser a 2° questão e até é em maioria rsrs.... tentei misturar com a 4° e eu diria que de certo 90%.

import pandas as pd
import matplotlib.pyplot as plt #faz os plots graficos
import seaborn as sns #auxlia o matplot com paleta de cores, não sabia ? agr você meu caro nobre ta sabendo
from tabulate import tabulate #faz a tabela

# Abre o set
arquivo = "/content/drive/MyDrive/archive/olist_order_items_dataset.csv" #Você que vai inspecionar meu código eu recomendo colocar o caminho de onde esta alocado os arquivos de set pois ao que parece o colabnão deixa as pessoas terem acesso ao caminho dod arquivos hospedado no meu drive.
set = pd.read_csv(arquivo, usecols=[2, 4, 5], encoding="utf-8")

# Aqui era as únicas datas que eu tinha, então considerei a como se fosse a data da venda
set['data'] = set['shipping_limit_date'].str.split(' ').str[0]

set['hora'] = pd.to_datetime(set['shipping_limit_date']).dt.strftime('%H:%M')

# Captura o pico de muitas e poucas vendas nas datas filtradas
contagem_por_data = set['data'].value_counts()
maior_pedidos = contagem_por_data.idxmax()
menor_pedidos = contagem_por_data.idxmin()

# Filtra os dados para os dias com o maior e o menor número de pedidos
set_maior_pedidos = set[set['data'] == maior_pedidos]
set_menor_pedidos = set[set['data'] == menor_pedidos]

# Cria o mapa de calor para os dias com o maior número de pedidos
set_maior_pedidos['hora'] = pd.to_datetime(set_maior_pedidos['shipping_limit_date']).dt.hour
heatmap_data_maior_pedidos = set_maior_pedidos.pivot_table(index='hora', aggfunc='size', fill_value=0)

# Cria o mapa de calor para os dias com o menor número de pedidos
set_menor_pedidos['hora'] = pd.to_datetime(set_menor_pedidos['shipping_limit_date']).dt.hour
heatmap_data_menor_pedidos = set_menor_pedidos.pivot_table(index='hora', aggfunc='size', fill_value=0)

#Mapa de calor
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

#Mapa de menor pedidos

sns.heatmap(pd.DataFrame(heatmap_data_maior_pedidos).transpose(), cmap='rocket_r', cbar=True, linewidths=0.5, linecolor='gray',
            annot=True, fmt='d', annot_kws={"size": 8}, square=True, vmin=0, vmax=max(heatmap_data_maior_pedidos.max(), heatmap_data_menor_pedidos.max()), ax=axes[0])
axes[0].set_title(f'Mapa de Calor - Distribuição de Pedidos ao Longo do Dia\nData com Maior Número de Pedidos: {maior_pedidos}')
axes[0].set_xlabel('Hora')
axes[0].set_ylabel('Número de Pedidos')

#Mapa de menor pedidos
sns.heatmap(pd.DataFrame(heatmap_data_menor_pedidos).transpose(), cmap='rocket_r', cbar=True, linewidths=0.5, linecolor='gray',
            annot=True, fmt='d', annot_kws={"size": 8}, square=True, vmin=0, vmax=max(heatmap_data_maior_pedidos.max(), heatmap_data_menor_pedidos.max()), ax=axes[1])
axes[1].set_title(f'Mapa de Calor - Distribuição de Pedidos ao Longo do Dia\nData com Menor Número de Pedidos: {menor_pedidos}')
axes[1].set_xlabel('Hora')
axes[1].set_ylabel('Número de Pedidos')

plt.tight_layout()
plt.show()

# Product ID mais pedido (Maior Número de Pedidos)
product_id_maior_pedidos = set_maior_pedidos['product_id'].value_counts().idxmax()

# Product ID mais pedido (Menor Número de Pedidos)
product_id_menor_pedidos = set_menor_pedidos['product_id'].value_counts().idxmax()

soma = set['price'].sum() #Soma o valor total da coluna price
div = soma / 112651 #Pega o valor total da coluna price e faz o calculo de ticket médio

#Faz a formatação para "R$", eu achei a automátca muito complicada
format1 = 'R$ {:,.2f}'.format(soma)
format2 = 'R$ {:,.2f}'.format(div)


#Faz o grid da tabela
tabela = [
    ["Valor Total", format1],
    ["Ticket Médio", format2],
    ["Data com Maior Número de Pedidos", maior_pedidos],
    ["Hora com Maior Número de Pedidos", set_maior_pedidos['hora'].value_counts().idxmax()],
    ["Data com Menor Número de Pedidos", menor_pedidos],
    ["Hora com Menor Número de Pedidos", set_menor_pedidos['hora'].value_counts().idxmax()],
    ["Product ID mais pedido (Maior Número de Pedidos)", product_id_maior_pedidos],
    ["Product ID mais pedido (Menor Número de Pedidos)", product_id_menor_pedidos]
]

# Tabela
print(tabulate(tabela, headers=["Descrição", "Dados"], tablefmt="github")) #tenho preferência por display mas aqui só ta rodando print

print('Dado esse conjunto de dados, observamos que o horário mais frequente para as vendas durante um dia de pico é por volta das 3 horas da madrugada. Além disso, a análise revela que o gasto médio por compra é de aproximadamente 120 reais. Em relação aos produtos, identificamos que o produto mais vendido durante esses períodos de alta demanda é o produto com o ID: 422879e10f46682990de24d770e7f83d. Essas informações podem ser valiosas para entender padrões de comportamento dos consumidores e orientar estratégias de negócios.')

In [ ]:
#Aqui é a 5° questão ou pelo menos o que era pra ser

import pandas as pd
from tabulate import tabulate
import plotly.graph_objects as go #faz a representação da tabela em tabela mais bonita vizualmente
from IPython.display import HTML


arquivo = "/content/drive/MyDrive/archive/olist_order_payments_dataset.csv" #Você que vai inspecionar meu código eu recomendo colocar o caminho de onde esta alocado os arquivos de set pois ao que parece o colabnão deixa as pessoas terem acesso ao caminho dod arquivos hospedado no meu drive.
set = pd.read_csv(arquivo, usecols=[2, 4], encoding="utf-8")

soma = set["payment_value"].sum() #Soma o valor total da coluna "payment_value"
div = soma / 103887 #Pega o valor total da coluna price e faz o calculo de ticket médio
faixas_de_preco = [(0, 50), (50, 100), (100, 200), (200, 500)] #Aqui irei trabalhar com uma pequena amostra de faixas de preço para demonstrar o método de pagamento mais utlizado nessas faixas

frequencias_por_faixa = {} #Contagens de cada tipo de pagamento

# Calcula com loops de verificação
for faixa in faixas_de_preco:
    faixa_inferior, faixa_superior = faixa
    subset = set[(set["payment_value"] >= faixa_inferior) & (set["payment_value"] < faixa_superior)]
    frequencias = subset["payment_type"].value_counts()
    frequencias_por_faixa[faixa] = frequencias

# Pagamento mais frequente em cada faixa de preço
tipos_de_pagamento_por_faixa = {}
for faixa, frequencias in frequencias_por_faixa.items():
    tipo_pagamento_mais_frequente = frequencias.idxmax()
    tipos_de_pagamento_por_faixa[faixa] = tipo_pagamento_mais_frequente

# Exibindo os resultados formatados
for faixa, tipo_pagamento in tipos_de_pagamento_por_faixa.items():
    faixa_formatada = f"R${faixa[0]} a R${faixa[1]} reais"

#Aqui vai criar um filtro para e "agrupar" tipo de pagamento
credit_card = set[set["payment_type"] == "credit_card"]
boleto = set[set["payment_type"] == "boleto"]
voucher = set[set["payment_type"] == "voucher"]
debit_card = set[set["payment_type"] == "debit_card"]

#Aqui vai calcular o total de vendas e o ticket médio por tipo de pagamento
valor_total_credit_card = credit_card["payment_value"].sum()
ticket_medio_credit_card = valor_total_credit_card / len(credit_card)

valor_total_boleto = boleto["payment_value"].sum()
ticket_medio_boleto = valor_total_boleto / len(boleto)

valor_total_voucher = voucher["payment_value"].sum()
ticket_medio_voucher = valor_total_voucher / len(voucher)

valor_total_debit_card = debit_card["payment_value"].sum()
ticket_medio_debit_card = valor_total_debit_card / len(debit_card)

#Faz a formatação para "R$", eu achei a automátca muito complicada
format1 = 'R$ {:,.2f}'.format(soma)
format2 = 'R$ {:,.2f}'.format(div)
format3 = 'R$ {:,.2f}'.format(valor_total_credit_card)
format4 = 'R$ {:,.2f}'.format(ticket_medio_credit_card)
format5 = 'R$ {:,.2f}'.format(valor_total_boleto)
format6 = 'R$ {:,.2f}'.format(ticket_medio_boleto)
format7 = 'R$ {:,.2f}'.format(valor_total_voucher)
format8 = 'R$ {:,.2f}'.format(ticket_medio_voucher)
format9 = 'R$ {:,.2f}'.format(valor_total_debit_card)
format10 = 'R$ {:,.2f}'.format(ticket_medio_debit_card)

# Dados da tabela
data = {
    "Descrição": ["Valor Total", "Ticket Médio", "Valor Total (Cartão de Crédito)",
                  "Ticket Médio (Cartão de Crédito)", "Valor Total (Boleto)",
                  "Ticket Médio (Boleto)", "Valor Total (Voucher)",
                  "Ticket Médio (Voucher)", "Valor Total (Cartão de Débito)",
                  "Ticket Médio (Cartão de Débito)"],
    "Dados": [format1, format2, format3, format4, format5, format6, format7,
              format8, format9, format10]
}

# Adicionando os dados de pagamento por faixa de preço na tabela
data["Descrição"].extend([
    "Pagamento mais usado na faixa de R$0 a R$50 reais",
    "Pagamento mais usado na faixa de R$50 a R$100 reais",
    "Pagamento mais usado na faixa de R$100 a R$200 reais",
    "Pagamento mais usado na faixa de R$200 a R$500 reais"
])

data["Dados"].extend([
    tipos_de_pagamento_por_faixa[(0, 50)],
    tipos_de_pagamento_por_faixa[(50, 100)],
    tipos_de_pagamento_por_faixa[(100, 200)],
    tipos_de_pagamento_por_faixa[(200, 500)]
])

# Cria a tabela
table = go.Figure(data=[go.Table(
    header=dict(values=["Descrição", "Dados"]),
    cells=dict(values=[data["Descrição"], data["Dados"]]))
])

# Converte em uma string html
html_table = table.to_html()

# vamos tentar quebrar uma linha
html_table = html_table.replace("< br >", " ")

# Exibi a tabela
display(HTML(html_table))
print("Após esta análise concluímos que os pagamentos com cartão de crédito representam 78,34% do valor de vendas totais e cartão de crédito é o meio de pagamento mais frequente em todas as faixas de preço consideradas. Isso sugere uma preferência dos clientes por esse meio de pagamento, independentemente do valor da compra.")

In [ ]:
#Aqui é a 6, uma tentativa.

import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML

arquivo = "/content/drive/MyDrive/archive/olist_sellers_dataset.csv"
set = pd.read_csv(arquivo)

# Converte nomes das colunas para strings
set.columns = set.columns.astype(str)

# Criar um dicionário para saber quais estados está atrelado a qual região
estado_para_regiao = {
    "AM": "Norte", "PA": "Norte", "RR": "Norte", "AP": "Norte", "RO": "Norte", "AC": "Norte", "TO": "Norte",
    "PI": "Nordeste", "MA": "Nordeste", "PE": "Nordeste", "RN": "Nordeste", "PB": "Nordeste", "CE": "Nordeste",
    "BA": "Nordeste", "AL": "Nordeste", "SE": "Nordeste",
    "MT": "Centro-Oeste", "MS": "Centro-Oeste", "GO": "Centro-Oeste",
    "SP": "Sudeste", "RJ": "Sudeste", "ES": "Sudeste", "MG": "Sudeste"
}

# Adicionar uma coluna 'regiao' ao set com a região de cada vendedor
set['regiao'] = set['seller_state'].map(estado_para_regiao)

# seller que mais vendeu no geral
seller_mais_vendeu_geral = set['seller_id'].value_counts().idxmax()
estado_seller_mais_vendeu = set.loc[set['seller_id'] == seller_mais_vendeu_geral, 'seller_state'].iloc[0]

# seller que mais vendeu por região
seller_mais_vendeu_por_regiao = set.groupby('regiao')['seller_id'].apply(lambda x: x.iloc[0]).reset_index()

# seller que mais vendeu por estado
seller_mais_vendeu_por_estado = set.groupby(['seller_state', 'seller_id']).size().groupby('seller_state').apply(lambda x: x.index[0][1]).reset_index()

# Converte os dados das tabelas para HTML
html_table_regiao = seller_mais_vendeu_por_regiao.to_html(index=False)
html_table_estado = seller_mais_vendeu_por_estado.to_html(index=False)

# Cria a tabela
table = go.Figure(data=[go.Table(
    header=dict(values=["Descrição", "Dados"]),
    cells=dict(values=[["Seller que mais vendeu no geral:", "Seller que mais vendeu por região:", "Seller que mais vendeu por estado:"],
                       [seller_mais_vendeu_geral + f" ({estado_seller_mais_vendeu})", html_table_regiao, html_table_estado]]))
])

# Converte em uma string HTML
html_table = table.to_html()

# Tenta quebrar uma linha
html_table = html_table.replace("< br >", " ")

# Exibe a tabela
HTML(html_table)

#não consigui fazer a conversão de string HTML de fornma correta e por isso saiu com tags na coluna dados
#consegui fazer de ultima hora, consegui emprrestar esse mac do meu amigo hehe.